In [11]:
import os
import numpy as np
import nibabel as nib
from skimage.transform import resize
import gzip
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

# Constants
target_shape = (32, 32, 32)
sequence_length = 20
future_steps = 5

# Paths
fmri_file_path = "D:/PROJECTS/fMRI_emotion/sub-14/func/sub-14_task-fe_bold.nii"
model_path = "D:/PROJECTS/emotion_detect/fmri_emotion_model_rnn_future.h5"  # Update with your actual model file path
label_encoder_path = "label_encoder.pkl"

# Load and preprocess a single fMRI file
def process_single_fmri_file(fmri_file_path, sequence_length, future_steps, target_shape):
    if fmri_file_path.endswith(".nii.gz"):
        with gzip.open(fmri_file_path, 'rb') as f_in:
            fmri_img = nib.FileHolder(fileobj=f_in)
            fmri_data = nib.Nifti1Image.from_file_map({'image': fmri_img}).get_fdata(dtype=np.float32)
    else:
        fmri_data = nib.load(fmri_file_path).get_fdata(dtype=np.float32)

    fmri_data = (fmri_data - fmri_data.min()) / (fmri_data.max() - fmri_data.min() + 1e-10)
    total_frames = fmri_data.shape[-1]

    if total_frames < sequence_length + future_steps:
        raise ValueError(f"fMRI file has {total_frames} frames, but {sequence_length + future_steps} are required!")

    num_sequences = total_frames - sequence_length - future_steps + 1
    sequences = np.zeros((num_sequences, sequence_length, *target_shape), dtype=np.float32)

    for i in range(num_sequences):
        seq = fmri_data[..., i:i + sequence_length]
        seq_resized = resize(seq, (*target_shape, sequence_length), anti_aliasing=True, preserve_range=True)
        sequences[i] = np.transpose(seq_resized, (3, 0, 1, 2))

    return sequences

# Load label encoder
with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

# Load model
model = load_model(model_path, compile= False)

# Preprocess input fMRI data
X_test = process_single_fmri_file(fmri_file_path, sequence_length, future_steps, target_shape)

# Predict class probabilities and decode to labels
pred_probs = model.predict(X_test)
pred_classes = np.argmax(pred_probs[:, -1], axis=1)
predicted_labels = label_encoder.inverse_transform(pred_classes)


# Display predicted labels
for i, label in enumerate(predicted_labels[:5]):
    print(f"Sample {i + 1} - Predicted: {label}")







6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 402ms/step
Sample 1 - Predicted: scrambled
Sample 2 - Predicted: scrambled
Sample 3 - Predicted: scrambled
Sample 4 - Predicted: scrambled
Sample 5 - Predicted: scrambled


In [10]:
for i in range(5):
    probs = pred_probs[i]
    label_probs = {
        label: float(prob.item()) if hasattr(prob, "item") else float(prob)
        for label, prob in zip(label_encoder.classes_, probs)
    }
    sorted_probs = dict(sorted(label_probs.items(), key=lambda x: x[1], reverse=True))
    print(f"\nSample {i+1} - Top Predictions:")
    for label, prob in sorted_probs.items():
        print(f"  {label}: {prob:.4f}")


ValueError: can only convert an array of size 1 to a Python scalar